## lgb多分类，将数据划分为四个城市，然后分开训练

In [2]:
import gc
import json
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from time import gmtime, strftime
import warnings
warnings.filterwarnings("ignore")

def split_od(data):
    data['o_lng'] = data['o'].apply(lambda x: float(x.split(',')[0])).astype(np.float16)
    data['o_lat'] = data['o'].apply(lambda x: float(x.split(',')[1])).astype(np.float16)
    data['d_lng'] = data['d'].apply(lambda x: float(x.split(',')[0])).astype(np.float16)
    data['d_lat'] = data['d'].apply(lambda x: float(x.split(',')[1])).astype(np.float16)
    return data

def city_flag(row):
    if row[1]>35:       # 北京
        return 1
    elif row[1]>27.5:   # 上海
        return 2
    elif row[1]<22.87 and row[0]>113.72:    # 深圳
        return 3
    else:    # 广州
        return 4

train_queries1 = pd.read_csv('../data/data_set_phase2/train_queries_phase1.csv')
train_queries2 = pd.read_csv('../data/data_set_phase2/train_queries_phase2.csv')
train_clicks1 = pd.read_csv('../data/data_set_phase2/train_clicks_phase1.csv')
train_clicks2 = pd.read_csv('../data/data_set_phase2/train_clicks_phase2.csv')
train_plans1 = pd.read_csv('../data/data_set_phase2/train_plans_phase1.csv')
train_plans2 = pd.read_csv('../data/data_set_phase2/train_plans_phase2.csv')
gc.collect()

train_queries = pd.concat([train_queries1, train_queries2], axis=0)
train_plans = pd.concat([train_plans1, train_plans2], axis=0)
train_clicks = pd.concat([train_clicks1, train_clicks2], axis=0)

test_plans = pd.read_csv('../data/data_set_phase2/test_plans.csv')
test_queries = pd.read_csv("../data/data_set_phase2/test_queries.csv")
profiles = pd.read_csv('../data/data_set_phase2/profiles.csv')

train_data = train_queries.merge(train_clicks, on='sid', how='left')
train_data = train_data.merge(train_plans, on='sid', how='left')
train_data = train_data.drop(['click_time'], axis=1)
train_data['click_mode'] = train_data['click_mode'].fillna(0)

test_data = test_queries.merge(test_plans, on='sid', how='left')
test_data['click_mode'] = -1

data = pd.concat([train_data, test_data], axis=0)
data = data.reset_index(drop=True)

print('total data size: {}'.format(data.shape))
print('raw data columns: {}'.format(', '.join(data.columns)))

data = split_od(data)
data['city_flag_o'] = data[['o_lng','o_lat']].apply(city_flag, axis=1)
data['city_flag_d'] = data[['d_lng','d_lat']].apply(city_flag, axis=1)

data = data[data['city_flag_o'] == data['city_flag_d']]
data = data.drop(['city_flag_d'],axis=1)

beijing = data[data['city_flag_o']==1]
shanghai = data[data['city_flag_o']==2]
shenzhen = data[data['city_flag_o']==3]
guangzhou = data[data['city_flag_o']==4]

beijing.to_csv('./processed/data_beijng.csv',index=False)
shanghai.to_csv('./processed/data_shanghai.csv',index=False)
shenzhen.to_csv('./processed/data_shenzhen.csv',index=False)
guangzhou.to_csv('./processed/data_guangzhou.csv',index=False)

total data size: (2304916, 8)
raw data columns: click_mode, d, o, pid, plan_time, plans, req_time, sid


In [3]:
def split_od(data):
    data['o_lng'] = data['o'].apply(lambda x: float(x.split(',')[0])).astype(np.float16)
    data['o_lat'] = data['o'].apply(lambda x: float(x.split(',')[1])).astype(np.float16)
    data['d_lng'] = data['d'].apply(lambda x: float(x.split(',')[0])).astype(np.float16)
    data['d_lat'] = data['d'].apply(lambda x: float(x.split(',')[1])).astype(np.float16)
    return data

def city_flag(row):
    if row[1]>35:       # 北京
        return 1
    elif row[1]>27.5:   # 上海
        return 2
    elif row[1]<22.87 and row[0]>113.72:    # 深圳
        return 3
    else:    # 广州
        return 4

train_queries1 = pd.read_csv('../data/data_set_phase2/train_queries_phase1.csv')
train_queries2 = pd.read_csv('../data/data_set_phase2/train_queries_phase2.csv')
train_clicks1 = pd.read_csv('../data/data_set_phase2/train_clicks_phase1.csv')
train_clicks2 = pd.read_csv('../data/data_set_phase2/train_clicks_phase2.csv')
train_plans1 = pd.read_csv('../data/data_set_phase2/train_plans_phase1.csv')
train_plans2 = pd.read_csv('../data/data_set_phase2/train_plans_phase2.csv')
gc.collect()

train_queries = pd.concat([train_queries1, train_queries2], axis=0)
train_plans = pd.concat([train_plans1, train_plans2], axis=0)
train_clicks = pd.concat([train_clicks1, train_clicks2], axis=0)

test_plans = pd.read_csv('../data/data_set_phase2/test_plans.csv')
test_queries = pd.read_csv("../data/data_set_phase2/test_queries.csv")
profiles = pd.read_csv('../data/data_set_phase2/profiles.csv')

train_data = train_queries.merge(train_clicks, on='sid', how='left')
train_data = train_data.merge(train_plans, on='sid', how='left')
train_data = train_data.drop(['click_time'], axis=1)
train_data['click_mode'] = train_data['click_mode'].fillna(0)

test_data = test_queries.merge(test_plans, on='sid', how='left')
test_data['click_mode'] = -1

data = pd.concat([train_data, test_data], axis=0)
data = data.reset_index(drop=True)

print('total data size: {}'.format(data.shape))
print('raw data columns: {}'.format(', '.join(data.columns)))

data = split_od(data)
data['city_flag_o'] = data[['o_lng','o_lat']].apply(city_flag, axis=1)
data['city_flag_d'] = data[['d_lng','d_lat']].apply(city_flag, axis=1)

shen_guang = data[data['city_flag_o']>2].reset_index(drop=True)

total data size: (2304916, 8)
raw data columns: click_mode, d, o, pid, plan_time, plans, req_time, sid


In [4]:
colla_filter = pd.read_csv('./xietong_feature_shenzhen.csv')
print(colla_filter.shape, shen_guang.shape)
gc.collect()

colla_filter = colla_filter.iloc[:,1:]
colla_col = colla_filter.columns.tolist()
shen_guang = pd.concat([shen_guang, colla_filter], axis=1)

print(shen_guang.shape)

shen_guang = shen_guang[shen_guang['city_flag_o'] == shen_guang['city_flag_d']]

shenzhen = shen_guang[shen_guang['city_flag_d']==3]
guangzhou = shen_guang[shen_guang['city_flag_d']==4]

print(shenzhen.shape, guangzhou.shape)


(1124625, 24) (1124625, 14)
(1124625, 37)
(555523, 37) (569040, 37)


FileNotFoundError: [Errno 2] No such file or directory: '../data/colla_filter_feat/colla_filter_shenzhen.csv'

In [5]:
shenzhen[colla_col].to_csv('./xietong_divide_shenzhen.csv', index=False)
guangzhou[colla_col].to_csv('./xietong_divide_guangzhou.csv', index=False)